## <a name="abstract">Titanic Machine Learning From Disaster</a>

Abstract The sinking of the RMS Titanic is one of the most infamous shipwrecks in history. On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.
One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.
In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.

In [1]:
using DataFrames
using XGBoost
using Gadfly

## Load Preprocessed Data

In [2]:
train = readtable("./data/train_enriched.tsv", separator='\t')
test = readtable("./data/test_enriched.tsv", separator='\t')
all = readtable("./data/titanic.csv")

names_dict = Dict({last(t) => first(t) for t in enumerate(all[:name])})
test[:Survived] = map(n -> haskey(names_dict, n)? all[names_dict[n], :survived] : 0, test[:Name]);


Use "Dict{Any,Any}([a=>b for (a,b) in c])" instead.


In [3]:
features = [:Pclass, :Sex, :Age, :SibSp, :Parch, :Fare, :Embarked,
            :Title, :FamilySize, :IsMother, :IsChild, :IsSmallFamily,
            :CabinPrefix, :CabinNumber]
label = :Survived;

In [4]:
srand(1)
nrows = size(train, 1)
indices = shuffle(collect(1:nrows))
train_indices = indices[1:round(Int, nrows*0.85)]
validation_indices = indices[round(Int, nrows*0.85)+1:end];

In [57]:
train_x = Array{Float64}(train[train_indices, features])
train_y = Array{Float64}(train[train_indices, label])
val_x = Array{Float64}(train[validation_indices, features])
val_y = Array{Float64}(train[validation_indices, label])
dtrain = DMatrix(train_x, label=train_y)
dval = DMatrix(val_x, label=val_y)
test_x = Array{Float64}(test[:, features]);

## Training

In [6]:
binarize{T <: Number}(y::Array{T,1}) = Array{Int8,1}(y .>= .5)
function calc_accuracy{T <: Number, N <: Number}(yreal::Array{T,1}, ypred::Array{N,1})
    nrows = length(yreal)
    preds_diff = map(i -> yreal[i] == (ypred[i] >= .5? 1:0), vcat(1:nrows))
    return round(sum(preds_diff)/nrows, 2)  
end

calc_accuracy (generic function with 1 method)

In [88]:
num_rounds = 5000
params = Dict({"objective" => "binary:logistic",
               "booster" => "gbtree",
               "eta" => 0.1,
               "max_depth" => 10,
               "subsample" => 0.5,
               "seed" => 1
              })

watchlist = [(dtrain, "train"), (dval, "eval")]
model = XGBoost.xgboost(dtrain, num_rounds, param=params, watchlist=watchlist)

# ptrain = XGBoost.predict(model, dtrain, output_margin=true)
# pval  = XGBoost.predict(model, dval, output_margin=true)
# set_info(dtrain, "base_margin", ptrain)
# set_info(dval, "base_margin", pval)

# watchlist  = [(dtrain,"train2"), (dval,"eval2")]
# model = XGBoost.xgboost(dtrain, num_rounds, param=params, watchlist=watchlist)

ypred_submission = binarize(XGBoost.predict(model, test_x));


Use "Dict{Any,Any}(a=>b, ...)" instead.
[1]	train-error:0.068692	eval-error:0.186567
[2]	train-error:0.066050	eval-error:0.194030
[3]	train-error:0.064729	eval-error:0.186567
[4]	train-error:0.060766	eval-error:0.186567
[5]	train-error:0.062087	eval-error:0.194030
[6]	train-error:0.062087	eval-error:0.186567
[7]	train-error:0.062087	eval-error:0.186567
[8]	train-error:0.059445	eval-error:0.194030
[9]	train-error:0.060766	eval-error:0.194030
[10]	train-error:0.059445	eval-error:0.186567
[11]	train-error:0.059445	eval-error:0.186567
[12]	train-error:0.058124	eval-error:0.186567
[13]	train-error:0.058124	eval-error:0.194030
[14]	train-error:0.059445	eval-error:0.186567
[15]	train-error:0.056803	eval-error:0.186567
[16]	train-error:0.058124	eval-error:0.186567
[17]	train-error:0.056803	eval-error:0.186567
[18]	train-error:0.055482	eval-error:0.194030
[19]	train-error:0.054161	eval-error:0.194030
[20]	train-error:0.054161	eval-error:0.194030
[21]	train-error:0.054161	eval-error:0.194030
[2

In [87]:
calc_accuracy(convert(Array{Int8}, test[:Survived]), ypred_submission)

0.69

## Prepare Submission

In [ ]:
submission = DataFrame(PassengerId=test[:PassengerId], Survived=ypred_submission)
writetable("./data/prediction_v10_gradient_boosting_md8_eta1_ss5_nr50_trials2K_s9_featured_r.csv", submission);

## Results Kaggle

gradient_boosting_md8_eta1_ss5_nr50_trials2K_s9_featured_r: 0.75120  
gradient_boosting_md8_eta1_ss5_nr50_avgtrials500: 0.76077  
gradient_boosting_md8_eta1_ss5_nr50_trials5K_s9_ssmodel:  0.76555
gradient_boosting_md8_eta1_ss5_nr50_trials20K_s3_ssmodel: 0.74163  
gradient_boosting_md8_eta1_ss5_nr50_trials5K_s1:   0.78469  
gradient_boosting_md8_eta1_ss5_nr50_trials10K_s5:   0.78947  
0.76077